##### Esercizio 5 lezione numero 12 del 29/10/2020

In [6]:
import pandas as pd
import numpy as np
pop = pd.read_csv(r'C:\Users\Laura\Desktop\PythonDataScienceHandbook-master\notebooks\data\state-population.csv')
areas = pd.read_csv(r'C:\Users\Laura\Desktop\PythonDataScienceHandbook-master\notebooks\data\state-areas.csv')
abbrevs = pd.read_csv(r'C:\Users\Laura\Desktop\PythonDataScienceHandbook-master\notebooks\data\state-abbrevs.csv')
display(pop.head() , areas.head() , abbrevs.head())

,state/region,ages,year,population
0,AL,under18,2012,1117489.0
1,AL,total,2012,4817528.0
2,AL,under18,2010,1130966.0
3,AL,total,2010,4785570.0
4,AL,under18,2011,1125763.0


,state,area (sq. mi)
0,Alabama,52423
1,Alaska,656425
2,Arizona,114006
3,Arkansas,53182
4,California,163707


,state,abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


L'obbiettivo è quello di classificare gli stati e i territori degli Stati Uniti in base alla densità di popolazione del 2010.
Combiniamo i set di dati per trovare il risultato.

Per prima cosa uniamo i dataframe pop e abbrevs in base alla colonna "stato / regione" di "pop" e alla colonna "abbreviation" di "abbrevs".

In [9]:
merged = pd.merge(pop, abbrevs, how='outer',left_on='state/region', right_on='abbreviation')
merged = merged.drop('abbreviation', 1) # eliminiamo la colonna abbrevation perchè è un informatizone duplicata
merged

,state/region,ages,year,population,state
0,AL,under18,2012,1117489.0,Alabama
1,AL,total,2012,4817528.0,Alabama
2,AL,under18,2010,1130966.0,Alabama
3,AL,total,2010,4785570.0,Alabama
4,AL,under18,2011,1125763.0,Alabama
...,...,...,...,...,...
2539,USA,total,2010,309326295.0,NaN
2540,USA,under18,2011,73902222.0,NaN
2541,USA,total,2011,311582564.0,NaN
2542,USA,under18,2012,73708179.0,NaN


Cerchiamo le righe con null per vedere se ci sono delle discrepanze

In [10]:
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state            True
dtype: bool

Ci sono delle informazioni sulla popolazione che sono nulle

In [11]:
merged[merged['population'].isnull()].head()

,state/region,ages,year,population,state
2448,PR,under18,1990,NaN,NaN
2449,PR,total,1990,NaN,NaN
2450,PR,total,1991,NaN,NaN
2451,PR,under18,1991,NaN,NaN
2452,PR,total,1993,NaN,NaN


I valori di popolazione nulli provengono da Porto Rico prima dell'anno 2000 e alcune delle nuove voci "state" sono nulle, il che significa che non c'era nessuna voce corrispondente nella chiave "abbrevs".

Le regioni a cui manca questa corrispondenza sono:

In [12]:
merged.loc[merged['state'].isnull(), 'state/region'].unique()

array(['PR', 'USA'], dtype=object)

il problema è che i dati i sulla popolazione includono voci per Porto Rico (PR) e gli Stati Uniti nel loro complesso (USA), mentre queste voci non compaiono nella chiave di abbreviazione dello stato.

Risolviamo il problema.

In [17]:
#Inseriamo nella variabile state in corrisponde del valore PR il valore Puerto Rico
merged.loc[merged['state/region'] == 'PR', 'state'] = 'Puerto Rico' 
#Inseriamo nella variabile state in corrisponde del valore USA il valore United States
merged.loc[merged['state/region'] == 'USA', 'state'] = 'United States'

#controlliamo se sono rimaste delle variabili null
merged.isnull().any()

state/region    False
ages            False
year            False
population       True
state           False
dtype: bool

Non ci sono più valori nulli nella colonna state, ora uniamo il risultato con il dataframe dell'area

In [22]:
final = pd.merge(merged, areas, on='state', how='left')
final

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0
...,...,...,...,...,...,...
2539,USA,total,2010,309326295.0,United States,NaN
2540,USA,under18,2011,73902222.0,United States,NaN
2541,USA,total,2011,311582564.0,United States,NaN
2542,USA,under18,2012,73708179.0,United States,NaN


Controlliamo i valori nulli per vedere se ci sono state discrepanze

In [23]:
final.isnull().any()

state/region     False
ages             False
year             False
population        True
state            False
area (sq. mi)     True
dtype: bool

Ci sono dei valori nulli nella colonna area

In [25]:
final['state'][final['area (sq. mi)'].isnull()].unique()

array(['United States'], dtype=object)

Il DataFrame non contiene l'intera area degli Stati Uniti.

Inseriamo il valore dell'area (usando la somma di tutte le aree statali), ma invece eliminiamo i valori nulli perché la densità di popolazione dell'intero Stati Uniti non è rilevante per ora.

In [26]:
final.dropna(inplace=True)
final

,state/region,ages,year,population,state,area (sq. mi)
0,AL,under18,2012,1117489.0,Alabama,52423.0
1,AL,total,2012,4817528.0,Alabama,52423.0
2,AL,under18,2010,1130966.0,Alabama,52423.0
3,AL,total,2010,4785570.0,Alabama,52423.0
4,AL,under18,2011,1125763.0,Alabama,52423.0
...,...,...,...,...,...,...
2491,PR,under18,2010,896945.0,Puerto Rico,3515.0
2492,PR,under18,2011,869327.0,Puerto Rico,3515.0
2493,PR,total,2011,3686580.0,Puerto Rico,3515.0
2494,PR,under18,2012,841740.0,Puerto Rico,3515.0


Selezioniamo la porzione di dati corrispondente all'anno 2010 e la popolazione totale.

In [37]:
data2010 = final.query("year == 2010 & ages == 'total'")
data2010.head()

,state/region,ages,year,population,state,area (sq. mi)
3,AL,total,2010,4785570.0,Alabama,52423.0
91,AK,total,2010,713868.0,Alaska,656425.0
101,AZ,total,2010,6408790.0,Arizona,114006.0
189,AR,total,2010,2922280.0,Arkansas,53182.0
197,CA,total,2010,37333601.0,California,163707.0


Calcoliamo la densità di popolazione, prima però reindicizzo i dati sullo stato

In [38]:
data2010.set_index('state', inplace=True) 
density = data2010['population'] / data2010['area (sq. mi)']

In [40]:
density.sort_values(ascending=False, inplace=True)
density.head()

state
District of Columbia    8898.897059
Puerto Rico             1058.665149
New Jersey              1009.253268
Rhode Island             681.339159
Connecticut              645.600649
dtype: float64

Il risultato è una classifica degli statiin ordine di densità di popolazione del 2010, in residenti per miglio quadrato.

Controllo anche la fine della lista.

In [41]:
density.tail()

state
South Dakota    10.583512
North Dakota     9.537565
Montana          6.736171
Wyoming          5.768079
Alaska           1.087509
dtype: float64

Lo stato meno denso è l'Alaska, con una media di poco più di un residente per miglio quadrato.